In [ ]:
#note_number = 21 # the 1st MIDI note corresponding to A0 (27.5Hz)
note_number = 69 # MIDI note corresponding to A4 (440Hz)
#note_number = 108 # the last MIDI note corresponding to C8 (4186Hz)

def set_piano(): 
    global piano 
    global selected_sf_name 
    global note_number 
    global originalAUDIO 
    global originalMIDI 
    global originalTXT
    global originalPNG
    global generatedAUDIO 
    global generatedMIDI 
    global generatedTXT 
    global generatedPNG
    global filename 
    global differencePNG 
    global differenceTXT 
    piano = "Yamaha_S6" 
    selected_sf_name = "Yamaha_S6" 
    originalAUDIO = "reference_note.wav" 
    originalMIDI = "reference_note.mid" 
    originalTXT = "reference_note.txt" 
    originalPNG = "reference_note.png" 
    generatedAUDIO = "generated_note.wav" 
    generatedMIDI = "generated_note.mid" 
    generatedTXT = "generated_note.txt" 
    generatedPNG = "generated_note.png" 
    filename = "differece_note_" + str(note_number) 
    differencePNG = filename + ".png" 
    differenceTXT = filename + ".txt" 
    
set_piano() 
print("piano = ", piano) 
print("note_nr = ", note_number) 
print("filename = ", filename )

from IPython.display import Audio

In [ ]:
# Creating a new MIDI file

from mido import Message, MidiFile, MidiTrack, MetaMessage

# Create a new MIDI file and add a track.
mid = MidiFile(ticks_per_beat=480)
track = MidiTrack()
mid.tracks.append(track)

# Set a tempo (here, 500000 microseconds per beat corresponds to 120 BPM).
# With 480 ticks per beat, one beat lasts 0.5 sec; thus, 240 ticks equal 0.25 sec.
track.append(MetaMessage('set_tempo', tempo=500000, time=0))

# Define our note and timing parameters.
note = note_number     # MIDI note number (middle C)
note_duration = 480  # Duration (in ticks) of each note (e.g. 120 ticks = 0.25 sec / 2)
gap_duration = 480   # Gap (in ticks) between the end of one note and the start of the next.
                     # This ensures that each note-on event occurs every 240 ticks (4 times per second).

# Generate 128 note events, with velocities from 0 to 127.
first_note = True
for velocity in range(128):
    if first_note:
        # The very first note_on event starts immediately.
        track.append(Message('note_on', note=note, velocity=velocity, time=0))
        first_note = False
    else:
        # For subsequent notes, delay the note_on event by gap_duration ticks.
        track.append(Message('note_on', note=note, velocity=velocity, time=gap_duration))
    # Each note is held for note_duration ticks before turning off.
    track.append(Message('note_off', note=note, velocity=0, time=note_duration))

# Save the MIDI file.
mid.save(originalMIDI)
print(f"MIDI file '{originalMIDI}' created successfully!")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pretty_midi

def visualize_piano_roll_from_midi():
    """
    Load a MIDI file ("input.mid"), extract note events, and visualize them as a piano roll.
    Each note is drawn as a rectangle from onset to offset time at the pitch corresponding to the MIDI note.
    The rectangle color intensity indicates the normalized note velocity.
    """
    # Load the MIDI file
    midi_data = pretty_midi.PrettyMIDI(originalMIDI)
    note_events = []

    # Iterate through all instruments and extract note events
    for instrument in midi_data.instruments:
        # Optionally, skip drum instruments if desired:
        # if instrument.is_drum:
        #     continue
        for note in instrument.notes:
            start = note.start
            end = note.end
            pitch = note.pitch
            # Normalize velocity (MIDI velocities are 0-127)
            velocity = note.velocity / 127.0  
            note_events.append([start, end, pitch, velocity])
    
    # Convert the list to a numpy array and ensure it's 2D
    note_events = np.atleast_2d(np.array(note_events))
    
    # Set up the plot
    fig, ax = plt.subplots(figsize=(12, 6))
    cmap = plt.cm.viridis  # colormap for velocity

    # Draw each note as a rectangle
    for note in note_events:
        onset, offset, midi_note, velocity = note
        duration = offset - onset
        color = cmap(velocity)
        rect = patches.Rectangle((onset, midi_note - 0.4), duration, 0.8,
                                 linewidth=1, edgecolor='black', facecolor=color)
        ax.add_patch(rect)
        
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("MIDI Note")
    ax.set_title("Piano Roll Visualization of " + originalMIDI)
    
    # Set x-axis limits based on note onsets and offsets
    time_min = np.min(note_events[:, 0]) - 0.5
    time_max = np.max(note_events[:, 1]) + 0.5
    ax.set_xlim(time_min, time_max)
    
    # Set y-axis limits based on MIDI note numbers
    midi_min = np.min(note_events[:, 2]) - 1
    midi_max = np.max(note_events[:, 2]) + 1
    ax.set_ylim(midi_min, midi_max)
    
    # Add a colorbar to indicate velocity
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=1))
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label("Normalized Velocity")
    
    plt.show()

# Call the function to visualize the piano roll from "input.mid"
visualize_piano_roll_from_midi()


In [ ]:
def midi_to_text_and_plot(midi_path, text_path, plot_path, title_text):
    """
    Decodes a MIDI file, writes its events into a text file, and plots:
    1) Velocity vs. Time (ignoring velocity 0) with Y-axis from 0 to 127
    2) Histogram of Velocity Values with 128 bins and X-axis from 0 to 127
    3) MIDI Note Numbers vs. Time mapped to piano keys (1-88)
    4) Histogram of MIDI Note Numbers (mapped to 1-88) with 88 bins
    Args:
        midi_path (str): Path to the input MIDI file.
        text_path (str): Path to save the output text file.
        plot_path (str): Path to save the generated plot as a PNG file.
        title_text (str): Text to be displayed in the upper left corner of the plot.
    """
    import mido
    import matplotlib.pyplot as plt

    midi_file = mido.MidiFile(midi_path)
    ticks_per_beat = midi_file.ticks_per_beat
    tempo = 500000  # Default 120 BPM tempo (500,000 microseconds per beat)

    # Store extracted data
    velocity_values = []
    velocity_times = []
    note_numbers = []
    note_times = []

    with open(text_path, 'w') as text_file:
        text_file.write(f"MIDI File: {midi_path}\n")
        text_file.write(f"Ticks per Beat: {ticks_per_beat}\n\n")

        absolute_ticks = 0  # Tracks cumulative time in ticks

        for i, track in enumerate(midi_file.tracks):
            text_file.write(f"Track {i}: {track.name}\n")
            text_file.write("-" * 40 + "\n")

            for msg in track:
                absolute_ticks += msg.time  # Convert relative to absolute time

                # Tempo Change Handling (Optional)
                if msg.type == 'set_tempo':
                    tempo = msg.tempo  # Set new tempo

                # Convert MIDI ticks to seconds
                seconds = mido.tick2second(absolute_ticks, ticks_per_beat, tempo)

                # Note-On Event (Velocity > 0)
                if msg.type == 'note_on' and msg.velocity > 0:
                    velocity_values.append(msg.velocity)  # Store velocity
                    velocity_times.append(seconds)         # Store time for velocity
                    note_numbers.append(msg.note)            # Store MIDI note number
                    note_times.append(seconds)               # Store time for note

                    text_file.write(f"Time {seconds:.3f} sec | NOTE_ON  | "
                                    f"Note: {msg.note:3d} | Velocity: {msg.velocity:3d}\n")

                # Note-Off Event
                elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
                    text_file.write(f"Time {seconds:.3f} sec | NOTE_OFF | "
                                    f"Note: {msg.note:3d} | Velocity: 0\n")

            text_file.write("\n")

    # Map MIDI note numbers to piano keys: MIDI 21 -> 1, MIDI 108 -> 88.
    mapped_note_numbers = [n - 20 for n in note_numbers]

    # Generate Plots as Subplots if there is any velocity data
    if velocity_values:
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))

        # Top Left: Velocity vs. Time (set Y-axis from 0 to 127)
        axes[0, 0].scatter(velocity_times, velocity_values, color='b', alpha=0.7, label="Note Velocity")
        axes[0, 0].set_xlabel("Time (seconds)")
        axes[0, 0].set_ylabel("Velocity (0-127)")
        axes[0, 0].set_title("Velocity vs. Time")
        axes[0, 0].legend()
        axes[0, 0].grid()
        axes[0, 0].set_ylim(0, 127)
        axes[0, 0].set_xlim(left=0)
        axes[0, 0].text(-0.07, 1.1, title_text, transform=axes[0, 0].transAxes, fontsize=12, verticalalignment='top')

        # Bottom Left: MIDI Note Numbers vs. Time (mapped to 1-88)
        axes[1, 0].scatter(note_times, mapped_note_numbers, color='purple', alpha=0.7, label="Piano Key")
        axes[1, 0].set_xlabel("Time (seconds)")
        axes[1, 0].set_ylabel("Piano Key (1-88)")
        axes[1, 0].set_title("MIDI Note Numbers vs. Time")
        axes[1, 0].legend()
        axes[1, 0].grid()
        axes[1, 0].set_xlim(left=0)
        axes[1, 0].set_ylim(1, 88)

        # Top Right: Histogram of Velocity Values (set X-axis from 0 to 127)
        axes[0, 1].hist(velocity_values, bins=128, range=(0, 127), color='g', alpha=0.7, edgecolor='black')
        axes[0, 1].set_xlabel("Velocity")
        axes[0, 1].set_ylabel("Number of Occurrences")
        axes[0, 1].set_title("Histogram of Velocity Values")
        axes[0, 1].grid()
        axes[0, 1].set_xlim(0, 127)

        # Bottom Right: Histogram of MIDI Note Numbers (mapped to 1-88)
        axes[1, 1].hist(mapped_note_numbers, bins=88, range=(1, 88), color='orange', alpha=0.7, edgecolor='black')
        axes[1, 1].set_xlabel("Piano Key (1-88)")
        axes[1, 1].set_ylabel("Number of Occurrences")
        axes[1, 1].set_title("Histogram of MIDI Note Numbers")
        axes[1, 1].grid()
        axes[1, 1].set_xlim(1, 88)

        plt.tight_layout()
        plt.savefig(plot_path)  # Save plot as PNG file
        plt.show()

    print(f"PLOTS saved to: {plot_path}")
    print(f"MIDI events saved to: {text_path}")

midi_to_text_and_plot(originalMIDI, originalTXT, originalPNG, "note_number: " + str(note_number))


In [ ]:
#rendering MIDI events with piano sounds

import subprocess
import ipywidgets as widgets
from IPython.display import display, Audio
import os


# Define available SoundFonts with your provided paths.
soundfonts = {
    "Mason&Hamlin": "/Users/user/.soundfonts/MasonHamlin-A-v7.sf2",
    "MuseScore": "/Users/user/.soundfonts/MuseScore_General.sf2",
    "4U_Steinway": "/Users/user/.soundfonts/4U-Steinway-v3.6.sf2",
    "Steinway_Chateau": "/Users/user/.soundfonts/Steinway-Chateau-Plus-Instruments-v1.7.sf2",
    "NY_S&S_Model_B": "/Users/user/.soundfonts/Dore Mark's NY S&S Model B-v5.2.sf2",
    "Nice_Steinway": "/Users/user/.soundfonts/Nice-Steinway-v3.9.sf2",
    "Fazioli_v2.5": "/Users/user/.soundfonts/Dore Mark's (SF) Fazioli-v2.5.sf2",
    "Fazioli_F308": "/Users/user/.soundfonts/Dore Mark's Fazioli F308-v3.0.sf2",
    "Yamaha_C5": "/Users/user/.soundfonts/Yamaha C5 Grand-v2.4.sf2",
    "Yamaha_S6": "/Users/user/.soundfonts/Dore Mark's Yamaha S6-v1.6.sf2",
    "Clementi_Fortepiano_1808": "/Users/user/.soundfonts/Clementi Fortepiano 1808 (Dore Mark)-v1.0.sf2"
}

# Define available MIDI files (update these paths as needed)
midi_files = {
#    "Step14": "Step_14.mid",
    "MIDI FILE": originalMIDI,
}

# Create radio buttons for SoundFont selection.
soundfont_selector = widgets.RadioButtons(
    options=list(soundfonts.keys()),
    description="SoundFont:",
    disabled=False
)

# Create radio buttons for MIDI file selection.
midi_selector = widgets.RadioButtons(
    options=list(midi_files.keys()),
    description="MIDI:",
    disabled=False
)

# Create a button widget to trigger synthesis.
synthesize_button = widgets.Button(
    description="Synthesize Audio",
    disabled=False,
    button_style='',  # Options: 'success', 'info', 'warning', 'danger'
    tooltip='Click to synthesize audio',
)

# Create an output widget to display messages.
output = widgets.Output()

def on_synthesize_button_clicked(b):
    global selected_sf_name  # Keep the variable global as requested.
    with output:
        output.clear_output()
        # Retrieve selected SoundFont and MIDI file paths.
        selected_sf_name = soundfont_selector.value
        selected_midi_name = midi_selector.value
        soundfont_path = soundfonts[selected_sf_name]
        midi_file_path = midi_files[selected_midi_name]
        
        # Verify that the files exist.
        if not os.path.exists(soundfont_path):
            print(f"SoundFont file not found: {soundfont_path}")
            return
        if not os.path.exists(midi_file_path):
            print(f"MIDI file not found: {midi_file_path}")
            return
        
        # Determine output audio file name.
        print("Selected SoundFont:", selected_sf_name)
        print("Selected MIDI:", selected_midi_name)
        print("Output audio file will be:", originalAUDIO)
        
        # Build the FluidSynth command.
        sample_rate = 44100
        command = [
            "fluidsynth",
            "-ni", 
            soundfont_path,
            midi_file_path,
            "-F", originalAUDIO,
            "-r", str(sample_rate)
        ]
        print("Running command:", " ".join(command))
        try:
            subprocess.run(command, check=True)
            print("Synthesis finished. Audio written to", originalAUDIO)
        except subprocess.CalledProcessError as e:
            print("Error during synthesis:", e)

# Bind the button click event to the callback function.
synthesize_button.on_click(on_synthesize_button_clicked)

# Display the widgets.
display(soundfont_selector, midi_selector, synthesize_button, output)

In [ ]:
# playing out 
Audio(originalAUDIO)

In [ ]:
# piano transcription from audio to MIDI using the AI model
!python example.py --audio_path={originalAUDIO} --output_midi_path={generatedMIDI}

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pretty_midi

def visualize_piano_roll_from_midi():
    """
    Load a MIDI file ("input.mid"), extract note events, and visualize them as a piano roll.
    Each note is drawn as a rectangle from onset to offset time at the pitch corresponding to the MIDI note.
    The rectangle color intensity indicates the normalized note velocity.
    """
    # Load the MIDI file
    midi_data = pretty_midi.PrettyMIDI(generatedMIDI)
    note_events = []

    # Iterate through all instruments and extract note events
    for instrument in midi_data.instruments:
        # Optionally, skip drum instruments if desired:
        # if instrument.is_drum:
        #     continue
        for note in instrument.notes:
            start = note.start
            end = note.end
            pitch = note.pitch
            # Normalize velocity (MIDI velocities are 0-127)
            velocity = note.velocity / 127.0  
            note_events.append([start, end, pitch, velocity])
    
    # Convert the list to a numpy array and ensure it's 2D
    note_events = np.atleast_2d(np.array(note_events))
    
    # Set up the plot
    fig, ax = plt.subplots(figsize=(12, 6))
    cmap = plt.cm.viridis  # colormap for velocity

    # Draw each note as a rectangle
    for note in note_events:
        onset, offset, midi_note, velocity = note
        duration = offset - onset
        color = cmap(velocity)
        rect = patches.Rectangle((onset, midi_note - 0.4), duration, 0.8,
                                 linewidth=1, edgecolor='black', facecolor=color)
        ax.add_patch(rect)
        
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("MIDI Note")
    ax.set_title("Piano Roll Visualization of " + generatedMIDI)
    
    # Set x-axis limits based on note onsets and offsets
    time_min = np.min(note_events[:, 0]) - 0.5
    time_max = np.max(note_events[:, 1]) + 0.5
    ax.set_xlim(time_min, time_max)
    
    # Set y-axis limits based on MIDI note numbers
    midi_min = np.min(note_events[:, 2]) - 1
    midi_max = np.max(note_events[:, 2]) + 1
    ax.set_ylim(midi_min, midi_max)
    
    # Add a colorbar to indicate velocity
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=0, vmax=1))
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label("Normalized Velocity")
    
    plt.show()

# Call the function to visualize the piano roll from "input.mid"
visualize_piano_roll_from_midi()


In [ ]:
def midi_to_text_and_plot(midi_path, text_path, plot_path, title_text):
    """
    Decodes a MIDI file, writes its events into a text file, and plots:
    1) Velocity vs. Time (ignoring velocity 0) with Y-axis from 0 to 127
    2) Histogram of Velocity Values with 128 bins and X-axis from 0 to 127
    3) MIDI Note Numbers vs. Time mapped to piano keys (1-88)
    4) Histogram of MIDI Note Numbers (mapped to 1-88) with 88 bins
    Args:
        midi_path (str): Path to the input MIDI file.
        text_path (str): Path to save the output text file.
        plot_path (str): Path to save the generated plot as a PNG file.
        title_text (str): Text to be displayed in the upper left corner of the plot.
    """
    import mido
    import matplotlib.pyplot as plt

    midi_file = mido.MidiFile(midi_path)
    ticks_per_beat = midi_file.ticks_per_beat
    tempo = 500000  # Default 120 BPM tempo (500,000 microseconds per beat)

    # Store extracted data
    velocity_values = []
    velocity_times = []
    note_numbers = []
    note_times = []

    with open(text_path, 'w') as text_file:
        text_file.write(f"MIDI File: {midi_path}\n")
        text_file.write(f"Ticks per Beat: {ticks_per_beat}\n\n")

        absolute_ticks = 0  # Tracks cumulative time in ticks

        for i, track in enumerate(midi_file.tracks):
            text_file.write(f"Track {i}: {track.name}\n")
            text_file.write("-" * 40 + "\n")

            for msg in track:
                absolute_ticks += msg.time  # Convert relative to absolute time

                # Tempo Change Handling (Optional)
                if msg.type == 'set_tempo':
                    tempo = msg.tempo  # Set new tempo

                # Convert MIDI ticks to seconds
                seconds = mido.tick2second(absolute_ticks, ticks_per_beat, tempo)

                # Note-On Event (Velocity > 0)
                if msg.type == 'note_on' and msg.velocity > 0:
                    velocity_values.append(msg.velocity)  # Store velocity
                    velocity_times.append(seconds)         # Store time for velocity
                    note_numbers.append(msg.note)            # Store MIDI note number
                    note_times.append(seconds)               # Store time for note

                    text_file.write(f"Time {seconds:.3f} sec | NOTE_ON  | "
                                    f"Note: {msg.note:3d} | Velocity: {msg.velocity:3d}\n")

                # Note-Off Event
                elif msg.type == 'note_off' or (msg.type == 'note_on' and msg.velocity == 0):
                    text_file.write(f"Time {seconds:.3f} sec | NOTE_OFF | "
                                    f"Note: {msg.note:3d} | Velocity: 0\n")

            text_file.write("\n")

    # Map MIDI note numbers to piano keys: MIDI 21 -> 1, MIDI 108 -> 88.
    mapped_note_numbers = [n - 20 for n in note_numbers]

    # Generate Plots as Subplots if there is any velocity data
    if velocity_values:
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))

        # Top Left: Velocity vs. Time (set Y-axis from 0 to 127)
        axes[0, 0].scatter(velocity_times, velocity_values, color='b', alpha=0.7, label="Note Velocity")
        axes[0, 0].set_xlabel("Time (seconds)")
        axes[0, 0].set_ylabel("Velocity (0-127)")
        axes[0, 0].set_title("Velocity vs. Time")
        axes[0, 0].legend()
        axes[0, 0].grid()
        axes[0, 0].set_ylim(0, 127)
        axes[0, 0].set_xlim(left=0)
        axes[0, 0].text(-0.07, 1.1, title_text, transform=axes[0, 0].transAxes, fontsize=12, verticalalignment='top')

        # Bottom Left: MIDI Note Numbers vs. Time (mapped to 1-88)
        axes[1, 0].scatter(note_times, mapped_note_numbers, color='purple', alpha=0.7, label="Piano Key")
        axes[1, 0].set_xlabel("Time (seconds)")
        axes[1, 0].set_ylabel("Piano Key (1-88)")
        axes[1, 0].set_title("MIDI Note Numbers vs. Time")
        axes[1, 0].legend()
        axes[1, 0].grid()
        axes[1, 0].set_xlim(left=0)
        axes[1, 0].set_ylim(1, 88)

        # Top Right: Histogram of Velocity Values (set X-axis from 0 to 127)
        axes[0, 1].hist(velocity_values, bins=128, range=(0, 127), color='g', alpha=0.7, edgecolor='black')
        axes[0, 1].set_xlabel("Velocity")
        axes[0, 1].set_ylabel("Number of Occurrences")
        axes[0, 1].set_title("Histogram of Velocity Values")
        axes[0, 1].grid()
        axes[0, 1].set_xlim(0, 127)

        # Bottom Right: Histogram of MIDI Note Numbers (mapped to 1-88)
        axes[1, 1].hist(mapped_note_numbers, bins=88, range=(1, 88), color='orange', alpha=0.7, edgecolor='black')
        axes[1, 1].set_xlabel("Piano Key (1-88)")
        axes[1, 1].set_ylabel("Number of Occurrences")
        axes[1, 1].set_title("Histogram of MIDI Note Numbers")
        axes[1, 1].grid()
        axes[1, 1].set_xlim(1, 88)

        plt.tight_layout()
        plt.savefig(plot_path)  # Save plot as PNG file
        plt.show()

    print(f"PLOTS saved to: {plot_path}")
    print(f"MIDI events saved to: {text_path}")
midi_to_text_and_plot(generatedMIDI, generatedTXT, generatedPNG, filename )


In [ ]:
#rendering MIDI events with piano sounds

import subprocess
import ipywidgets as widgets
from IPython.display import display, Audio
import os


# Define available SoundFonts with your provided paths.
soundfonts = {
    "Mason&Hamlin": "/Users/user/.soundfonts/MasonHamlin-A-v7.sf2",
    "MuseScore": "/Users/user/.soundfonts/MuseScore_General.sf2",
    "4U_Steinway": "/Users/user/.soundfonts/4U-Steinway-v3.6.sf2",
    "Steinway_Chateau": "/Users/user/.soundfonts/Steinway-Chateau-Plus-Instruments-v1.7.sf2",
    "NY_S&S_Model_B": "/Users/user/.soundfonts/Dore Mark's NY S&S Model B-v5.2.sf2",
    "Nice_Steinway": "/Users/user/.soundfonts/Nice-Steinway-v3.9.sf2",
    "Fazioli_v2.5": "/Users/user/.soundfonts/Dore Mark's (SF) Fazioli-v2.5.sf2",
    "Fazioli_F308": "/Users/user/.soundfonts/Dore Mark's Fazioli F308-v3.0.sf2",
    "Yamaha_C5": "/Users/user/.soundfonts/Yamaha C5 Grand-v2.4.sf2",
    "Yamaha_S6": "/Users/user/.soundfonts/Dore Mark's Yamaha S6-v1.6.sf2",
    "Clementi_Fortepiano_1808": "/Users/user/.soundfonts/Clementi Fortepiano 1808 (Dore Mark)-v1.0.sf2"
}

# Define available MIDI files (update these paths as needed)
midi_files = {
#    "Step14": "Step_14.mid",
    "MIDI FILE": generatedMIDI,
}

# Create radio buttons for SoundFont selection.
soundfont_selector = widgets.RadioButtons(
    options=list(soundfonts.keys()),
    description="SoundFont:",
    disabled=False
)

# Create radio buttons for MIDI file selection.
midi_selector = widgets.RadioButtons(
    options=list(midi_files.keys()),
    description="MIDI:",
    disabled=False
)

# Create a button widget to trigger synthesis.
synthesize_button = widgets.Button(
    description="Synthesize Audio",
    disabled=False,
    button_style='',  # Options: 'success', 'info', 'warning', 'danger'
    tooltip='Click to synthesize audio',
)

# Create an output widget to display messages.
output = widgets.Output()

def on_synthesize_button_clicked(b):
    global selected_sf_name  # Keep the variable global as requested.
    with output:
        output.clear_output()
        # Retrieve selected SoundFont and MIDI file paths.
        selected_sf_name = soundfont_selector.value
        selected_midi_name = midi_selector.value
        soundfont_path = soundfonts[selected_sf_name]
        midi_file_path = midi_files[selected_midi_name]
        
        # Verify that the files exist.
        if not os.path.exists(soundfont_path):
            print(f"SoundFont file not found: {soundfont_path}")
            return
        if not os.path.exists(midi_file_path):
            print(f"MIDI file not found: {midi_file_path}")
            return
        
        # Determine output audio file name.
        print("Selected SoundFont:", selected_sf_name)
        print("Selected MIDI:", selected_midi_name)
        print("Output audio file will be:", generatedAUDIO)
        
        # Build the FluidSynth command.
        sample_rate = 44100
        command = [
            "fluidsynth",
            "-ni", 
            soundfont_path,
            midi_file_path,
            "-F", generatedAUDIO,
            "-r", str(sample_rate)
        ]
        print("Running command:", " ".join(command))
        try:
            subprocess.run(command, check=True)
            print("Synthesis finished. Audio written to", generatedAUDIO)
        except subprocess.CalledProcessError as e:
            print("Error during synthesis:", e)

# Bind the button click event to the callback function.
synthesize_button.on_click(on_synthesize_button_clicked)

# Display the widgets.
display(soundfont_selector, midi_selector, synthesize_button, output)

In [ ]:
# playing out
Audio(generatedAUDIO)

In [ ]:
# comparing original velocities with those generated by the AI model and calculating Best Fit to adjust their values

import mido
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText

def extract_note_data(midi_filename):
    """
    Loads a MIDI file and extracts absolute times (in seconds), velocities, 
    and MIDI note numbers for all note_on events (with nonzero velocity). 
    The times are shifted so that the first event occurs at 0 seconds.
    
    Args:
        midi_filename (str): Path to the MIDI file.
        
    Returns:
        times (list of float): Times in seconds (shifted so first event is at 0).
        velocities (list of int): Velocity values (0-127) for each note_on event.
        notes (list of int): MIDI note numbers for each note_on event.
    """
    mid = mido.MidiFile(midi_filename)
    ticks_per_beat = mid.ticks_per_beat

    # Get tempo from the first track; default to 500000 µs per beat (120 BPM) if not found.
    tempo = 500000
    for msg in mid.tracks[0]:
        if msg.type == 'set_tempo':
            tempo = msg.tempo
            break

    absolute_tick = 0
    times = []
    velocities = []
    notes = []
    
    # Iterate over messages in time order, accumulating delta times (in ticks).
    for msg in mid:
        absolute_tick += msg.time
        if msg.type == 'note_on' and msg.velocity > 0:
            time_sec = mido.tick2second(absolute_tick, ticks_per_beat, tempo)
            times.append(time_sec)
            velocities.append(msg.velocity)
            notes.append(msg.note)
    
    # Shift times so that the first event is at 0 seconds.
    if times:
        t0 = times[0]
        times = [t - t0 for t in times]
        
    return times, velocities, notes
    
# Specify your two MIDI files.
midi_filename1 =  generatedMIDI
midi_filename2 = originalMIDI

# Extract note data from both MIDI files.
times1, velocities1, notes1 = extract_note_data(midi_filename1)
times2, velocities2, notes2 = extract_note_data(midi_filename2)

# Use the same number of events from both files.
n1 = len(times1)
n2 = len(times2)
n = min(n1, n2)

# Instead of using time stamps, we create a common x-axis of note indices (0 to n-1)
common_indices = np.arange(n)
velocities1_plot = np.array(velocities1[:n])
velocities2_plot = np.array(velocities2[:n])

# Calculate the difference (generated - reference) for each event.
velocity_diff_ref = velocities1_plot - velocities2_plot

# Calculate the best fit line (y = A*x + B) for the generated_note.mid velocities.
A, B = np.polyfit(common_indices, velocities1_plot, 1)
y_fit = A * common_indices + B

# New difference computed as (best fit - reference).
velocity_diff_ref_fit = y_fit - velocities2_plot

# Calculate the fourth chart using subtraction: (generated - (best fit - reference))
fourth_values = velocities1_plot - velocity_diff_ref_fit

# Create subplots: the upper subplot for the line plots and the lower for the bar charts.
#fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6), sharex=True)

# --- Upper subplot ---
# Plot reference velocities (black).
ax1.plot(common_indices, velocities2_plot, color='black', marker='o', linestyle='-', label=midi_filename2)
# Plot generated velocities (blue).
ax1.plot(common_indices, velocities1_plot, color='blue', marker='o', linestyle='-', label=midi_filename1)
# Plot the best-fit line (red).
ax1.plot(common_indices, y_fit, color='red', linewidth=2, label='Best Fit (generated)')
# Plot the new fourth chart (green) using subtraction with the updated label.
ax1.plot(common_indices, fourth_values, color='green', marker='o', linestyle='-', 
         label='postprocessed (generated - Best Fit + reference)')

ax1.set_ylabel('Velocity (MIDI)')
note_nr = notes1[0] if notes1 else "N/A"
#ax1.set_title("MIDI Note Nr = " + str(note_nr) + " reference, generated, best fit, and postprocessed velocities")
ax1.set_title(f"MIDI Note Nr = {note_number} reference, generated, best fit, and postprocessed velocities")

ax1.legend()
ax1.grid(True)

# Display the calculated coefficients A and B in the lower right corner.
anchored_text = AnchoredText(
    f"Best fit correction: Y = {A:.2f}X + {B:.2f}\nA = {A:.2f}\nB = {B:.2f}",
    loc='lower right',
    prop=dict(size=12, ha='left')
)
anchored_text.patch.set_boxstyle("round,pad=0.5")
anchored_text.patch.set_facecolor('white')
anchored_text.patch.set_alpha(0.5)
ax1.add_artist(anchored_text)

# --- Lower subplot ---
# We'll use a bar width that allows two bars per index.
bar_width = 0.35
x_left = common_indices - bar_width/2   # positions for blue bars: generated - reference
x_right = common_indices + bar_width/2  # positions for red bars: best fit - reference

# Bar chart 1: Difference between generated and reference velocities (blue).
ax2.bar(x_left, velocity_diff_ref, width=bar_width, color='blue', alpha=0.7, label='Generated - Reference')
# Bar chart 2: Difference between best fit and reference (red).
ax2.bar(x_right, velocity_diff_ref_fit, width=bar_width, color='red', alpha=0.7, label='Best Fit - Reference')

# Change the xlabel for the lower subplot to "Velocity (0:127)".
ax2.set_xlabel('Velocity (0:127)')
ax2.set_ylabel('Velocity Difference')
ax2.set_title("Differences in Velocities")
ax2.grid(True)
ax2.legend()

plt.tight_layout()

# Add a label "qqq" in the lower left corner of the entire figure.
plt.figtext(0.01, 0.01, "Piano:  " + piano + "", ha="left", va="bottom", fontsize=12,
            bbox=dict(facecolor='white', alpha=0.5, pad=5))


# Save the plot as a PNG image.
plt.savefig(differencePNG)
print("Plot saved as " + differencePNG)

# Save the calculated differences to a text file.
with open(differenceTXT, "w") as f:
    f.write("Reference\tGenerated\tDiff (Generated - Reference)\tDiff (Best Fit - Reference)\n")
    #for idx, d_ref, d_fit in zip(common_indices, velocity_diff_ref, velocity_diff_ref_fit):
    for idx, gen_note, d_ref, d_fit in zip(common_indices, velocities1_plot, velocity_diff_ref, velocity_diff_ref_fit):
        f.write(f"{idx+1}\t{gen_note}\t{d_ref}\t{d_fit}\n")
print("Calculated differences saved in " + differenceTXT)

plt.show()
